In [ ]:
import numpy as np
import pandas as pd

import torch

import math

##__1.__ hit rate at k

In [ ]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [ ]:
len(recommended_list)

10

In [ ]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = int(flags.sum() > 0) * 1
    
    return hit_rate

In [ ]:
hit_rate_at_k(recommended_list, bought_list)

1

##__2.__ money_precision_at_k

In [ ]:
prices_recommended = np.random.randint(0, 1000, len(recommended_list))
prices_recommended

array([ 69,   0,  81, 284, 540, 845, 528, 583, 244, 285])

In [ ]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = (prices_recommended @ flags) / prices_recommended.sum()   
    
    return precision

In [ ]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.3624229979466119

##__3.__ recall_at_k

In [ ]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [ ]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:5])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [ ]:
recall_at_k(recommended_list, bought_list, k=5)

0.5

##__4.__ money_recall_at_k

In [ ]:
prices_bought = np.random.randint(0, 1000, len(bought_list))
prices_bought

array([287, 888, 403, 609])

In [ ]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:5])
    prices_recommended = np.array(prices_recommended[:k])
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list)

    recall = (flags @ prices_recommended) / prices_bought.sum()

    return recall

In [ ]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.16140832190214907

##__5.__ map@k

In [ ]:
recommended_list = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],] #id товаров
bought_list = [[521, 32, 143, 991], [146, 29]]

In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [ ]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [ ]:
def map_k(recommended_list, bought_list, k=5, u=1):
    
    u = len(recommended_list)
    
    sum = 0
    
    for i in range(u):
        sum += ap_k(recommended_list[i], bought_list[i], k)
    
    sum /= u
    
    return sum

In [ ]:
map_k(recommended_list, bought_list, k=5, u=1)

0.875

##__7.__ * NDCG@k - для оценок пользователей (1-3) по учебному примеру из лекции (можно не в pyTorch, а в Numpy - в лекции пример массивов)

In [ ]:
N = 5
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

In [ ]:
ys_true

tensor([2, 3, 2, 2, 1])

In [ ]:
ys_pred

tensor([0.3806, 0.7246, 0.2737, 0.8824, 0.2433])

In [ ]:
# итоговый результат модели - отсортированный ys_pred
ys_pred

tensor([0.3806, 0.7246, 0.2737, 0.8824, 0.2433])

In [ ]:
torch.sort(ys_pred, descending=True, dim=0)

torch.return_types.sort(values=tensor([0.8824, 0.7246, 0.3806, 0.2737, 0.2433]), indices=tensor([3, 1, 0, 2, 4]))

In [ ]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == "exp2":
        gain = 2 ** y_value - 1
    elif gain_scheme == "const":
        gain = y_value
    else:
        raise ValueError(f"{gain_scheme} method not supported, only exp2 and const.")
    return float(gain)

In [ ]:
def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    ret = 0
    for idx, cur_y in enumerate(ys_true_sorted, 1):
        gain = compute_gain(cur_y, gain_scheme)
        if idx > 2:
            gain /= math.log2(idx)
        
        ret += gain
    
    ret /= len(ys_true)
    return ret

In [ ]:
dcg(ys_true, ys_pred, "exp2")

2.7646931637575536

In [ ]:
dcg(ys_true, ys_pred, "const")

1.5385072130432618